In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [5]:
import os
import pandas as pd

file_path = '/content/gdrive/My Drive/retailrocket-recommender-system-dataset'
os.listdir(file_path)

#读入event.csv文件
events = pd.read_csv(os.path.join(file_path, 'events.csv'))
print('Event data\n', events.head())

print('Unique counts: ', events.nunique())

print('Kind of events: ', events.event.unique())

Event data
        timestamp  visitorid event  itemid  transactionid
0  1433221332117     257597  view  355908            NaN
1  1433224214164     992329  view  248676            NaN
2  1433221999827     111016  view  318965            NaN
3  1433221955914     483717  view  253185            NaN
4  1433221337106     951259  view  367447            NaN
Unique counts:  timestamp        2750455
visitorid        1407580
event                  3
itemid            235061
transactionid      17672
dtype: int64
Kind of events:  ['view' 'addtocart' 'transaction']


In [6]:
#将三种不同事件用数值表示
events.event.replace(to_replace = dict(view = 1, addtocart = 2,
                                    transaction = 3), inplace = True)

print(events.head())

       timestamp  visitorid  event  itemid  transactionid
0  1433221332117     257597      1  355908            NaN
1  1433224214164     992329      1  248676            NaN
2  1433221999827     111016      1  318965            NaN
3  1433221955914     483717      1  253185            NaN
4  1433221337106     951259      1  367447            NaN


In [0]:

#删除分析不需要的数据项
              
events.drop(['transactionid'], axis = 1, inplace = True)
events.drop(['timestamp'], axis = 1, inplace = True)

In [8]:
#将数据划分为训练集，校验集合测试集
split_1 = int(0.8 * len(events))
split_2 = int(0.9 * len(events))
train = events[: split_1]
valid = events[split_1 : split_2]
test = events[split_2: ]

print(train.event.shape)

(2204880,)


In [7]:
#客户数量
n_visitors = events.visitorid.nunique()
#商品数量
n_items = events.itemid.nunique()

n_latent_factors = 5

import keras
from keras.layers import Input, Embedding, Flatten, Dense
from keras.models import Model

item_input = Input(shape = [1], name = 'Items')

'''
我们用长度为5的向量来对编码每个客户以及每件商品，这与我们前面分析好坏影评时，
用向量来编码每个单词是同样道理
'''
item_embed = Embedding(n_items + 1, n_latent_factors, name = 'ItemsEmbedding')(item_input)
item_vec = Flatten(name = 'ItemFlatten')(item_embed)

visitor_input = Input(shape = [1], name = 'visitor')
visitor_embed = Embedding(n_visitors + 1, 
                         n_latent_factors,
                         name = 'visitorsEmbedding')(visitor_input)
visitor_vec = Flatten(name = 'VisitorFlatten')(visitor_embed)

concat = keras.layers.concatenate([item_vec, visitor_vec], name = 'Concat')
fc_1 = Dense(80, name = 'FC-1')(concat)
fc_2 = Dense(40, name = 'FC-2')(fc_1)
fc_3 = Dense(20, name = 'FC-3', activation = 'relu')(fc_2)

output = Dense(1, activation = 'relu', name = 'Output')(fc_3)

optimizer = keras.optimizers.Adam(lr = 0.001)
model = keras.Model([item_input, visitor_input], output)
model.compile(optimizer = optimizer, loss = 'mse')

model.summary()

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Items (InputLayer)              (None, 1)            0                                            
__________________________________________________________________________________________________
visitor (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
ItemsEmbedding (Embedding)      (None, 1, 5)         1175310     Items[0][0]                      
__________________________________________________________________________________________________
visitorsEmbedding (Embedding)   (None, 1, 5)         7037905     visitor[0][0]                    
_____________________________________

In [10]:
from keras.models import load_model
model = load_model('/content/gdrive/My Drive/product_recommentation/recommend_model.h5')

model.fit([train.visitorid, train.itemid], train.event, epochs = 10)
score = model.evaluate([test.visitorid, test.itemid], test.event)
print('mean square error: ', score)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
2204880/2204880 [==============================] - 984s 446us/step - loss: 0.0598
Epoch 2/10
2204880/2204880 [==============================] - 1014s 460us/step - loss: 0.3221
Epoch 3/10
2204880/2204880 [==============================] - 1017s 461us/step - loss: 0.0927
Epoch 4/10
2204880/2204880 [==============================] - 1016s 461us/step - loss: 0.0655
Epoch 5/10
2204880/2204880 [==============================] - 1015s 461us/step - loss: 13.8046
Epoch 6/10
2204880/2204880 [==============================] - 1018s 462us/step - loss: 0.0955
Epoch 7/10
2204880/2204880 [==============================] - 1021s 463us/step - loss: 0.0580
Epoch 8/10
2204880/2204880 [==============================] - 980s 444us/step - loss: 0.0566
Epoch 9/10
2204880/2204880 [==============================] - 933s 423us/step - loss: 0.0566
Epoch 10/10
275611/275611 [==============================] - 15s 53us/step


NameError: ignored

In [0]:
!mkdir '/content/gdrive/My Drive/product_recommentation'

In [0]:
save_path = '/content/gdrive/My Drive/product_recommentation/recommend_model.h5'
model.save(save_path)

In [9]:
from keras.models import load_model
model = load_model('/content/gdrive/My Drive/product_recommentation/recommend_model.h5')

score = model.evaluate([test.visitorid, test.itemid], test.event)
print('mean square error: ', score)

275611/275611 [==============================] - 18s 66us/step
mean square error:  0.0818186502982947
